In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose

import sys


In [2]:

#import custome modules
sys.path.append('../')

from scripts.get_missing_information import MissingInformation
from scripts.get_dataframe_information import DataFrameInformation
from scripts.ploting_utils import Plotters
from scripts.data_clean_handler import CleanData
from scripts import data_loader

from scripts.dvc_data_fetch import DataLoader

dvc_load = DataLoader()

cleaner = CleanData()
minfo = MissingInformation()
dinfo = DataFrameInformation()
pltu = Plotters(6,4)

## Fetch the dataframes from DVC

In [3]:

# First load the cleaned stores data
data_path = 'data/cleaned/store.csv'
version = 'store_v2'
repo = '../'

store_df = dvc_load.dvc_get_data(data_path, version, repo)

# Then load the raw sales data
data_path = 'data/merged/train.csv'
version = 'train_v2'
repo = '../'

train_df = dvc_load.dvc_get_data(data_path, version, repo)

# Finally load the test data
data_path = 'data/merged/test.csv'
version = 'test_v2'
repo = '../'

test_df = dvc_load.dvc_get_data(data_path, version, repo)


DataLoaderLogger - INFO - DVC: CSV file read with path: data/cleaned/store.csv | version: store_v2 | from: ../
DataLoaderLogger - INFO - DVC: CSV file read with path: data/merged/train.csv | version: train_v2 | from: ../
DataLoaderLogger - INFO - DVC: CSV file read with path: data/merged/test.csv | version: test_v2 | from: ../


## Features correlation to target

In [4]:
train_df.corr()["Sales"].sort_values(ascending=False)


Sales                        1.000000
Customers                    0.894711
Open                         0.678472
Promo                        0.452345
SalesPerCustomer             0.186581
SchoolHoliday                0.085124
Promo2SinceWeek              0.059558
WeekOfYear                   0.052946
Month                        0.048768
Year                         0.023519
CompetitionOpenSinceYear     0.012659
Store                        0.005126
Promo2SinceYear             -0.021127
CompetitionDistance         -0.025273
CompetitionOpenSinceMonth   -0.028257
Promo2                      -0.091040
DayOfWeek                   -0.462125
Name: Sales, dtype: float64

In [5]:
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         41088 non-null  int64  
 1   Store                      41088 non-null  int64  
 2   DayOfWeek                  41088 non-null  int64  
 3   Date                       41088 non-null  object 
 4   Open                       41077 non-null  float64
 5   Promo                      41088 non-null  int64  
 6   StateHoliday               41088 non-null  object 
 7   SchoolHoliday              41088 non-null  int64  
 8   StoreType                  41088 non-null  object 
 9   Assortment                 41088 non-null  object 
 10  CompetitionDistance        41088 non-null  float64
 11  CompetitionOpenSinceMonth  25872 non-null  float64
 12  CompetitionOpenSinceYear   25872 non-null  float64
 13  Promo2                     41088 non-null  int